In [ ]:
import numpy as np
import pandas as pd
import json, nltk
import spacy
import re

In [ ]:
"""nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')"""

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
total_data = pd.read_csv("data.csv")

In [ ]:
total_data['text']=total_data['text'].apply(str)
total_data = total_data.dropna()

In [ ]:
!pip install emot

In [ ]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
# Converting emojis to words
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
        return text
# Converting emoticons to words    
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
        return text

In [ ]:
def process_tweet(tweet):
    tweet = tweet.lower()                                             # Lowercases the string
    tweet = re.sub('@[^\s]+', '', tweet)                              # Removes usernames
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', tweet)   # Remove URLs
    tweet = re.sub(r"\d+", " ", str(tweet))                           # Removes all digits
    tweet = re.sub('&quot;'," ", tweet)                               # Remove (&quot;) 
    tweet = convert_emojis(tweet)  
    tweet = convert_emoticons(tweet)                                             # Replaces Emojis
    tweet = re.sub(r"\b[a-zA-Z]\b", "", str(tweet))                   # Removes all single characters
    for word in tweet.split():
        if word.lower() in contractions:
            tweet = tweet.replace(word, contractions[word.lower()])   # Replaces contractions
    tweet = re.sub(r"[^\w\s]", " ", str(tweet))                       # Removes all punctuations
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)                         # Convert more than 2 letter repetitions to 2 letter
    tweet = re.sub(r"\s+", " ", str(tweet))                           # Replaces double spaces with single space    
    return tweet

In [ ]:
total_data['text'] = np.vectorize(process_tweet)(total_data['text'])

In [ ]:
"""#Importing stopwords from nltk library
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# Function to remove the stopwords
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# Applying the stopwords to 'text_punct' and store into 'text_stop'
total_data["text"] = total_data["text"].apply(stopwords)
total_data["text"].head()"""

In [ ]:
"""# Checking the first 10 most frequent words
from collections import Counter
cnt = Counter()
for text in total_data["text"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)"""

In [ ]:
"""# Removing the frequent words
freq = set([w for (w, wc) in cnt.most_common(10)])

# function to remove the frequent words
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not in freq])

# Passing the function freqwords
total_data["text"] = total_data["text"].apply(freqwords)
total_data["text"].head()"""

In [ ]:
"""# Removal of 10 rare words 
freq = pd.Series(' '.join(total_data['text']).split()).value_counts()[-10:] # 10 rare words
freq = list(freq.index)
total_data['text'] = total_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
total_data['text'].head()"""

In [ ]:
#spell check
from textblob import TextBlob
total_data['text'].apply(lambda x: str(TextBlob(x).correct()))
total_data.head(10)

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import en_core_web_lg
# call model
ner = en_core_web_lg.load()

In [ ]:
final_text = []
for index, row in total_data.iterrows():
  text_data = row['text']
  t = ner(text_data)
  final_text.append(" ".join([ent.text for ent in t if not ent.ent_type_]))

df = pd.DataFrame()
df = total_data

df['text'] = final_text

In [ ]:
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
df = df.dropna()

In [ ]:
def label(df):
  if (df['polarity']>=-0.2) and (df['polarity']<=0.2):
    return 0
  elif (df['polarity']>0.2):
    return 1
  elif (df['polarity']<-0.2):
    return 2

In [ ]:
def label(df):
  if (df['subjectivity']>0.5) :
    return 1
  elif (df['subjectivity']<=0.5) :
    return 0

In [ ]:
df['label'] = df.apply(label, axis=1)

In [ ]:
# Train test split
train, test = train_test_split(df, test_size=0.2, random_state=0)

In [ ]:
!pip install transformers

In [ ]:
# Loading tokenizer and encoding our data
from transformers import DistilBertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    train.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=64, 
    return_tensors='pt'
)

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=64, 
    return_tensors='pt'
)

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train.label.values)

In [ ]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df.label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

In [ ]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
# BERT pretrained model
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3, output_attentions=False, output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
# Creating Dataloaders
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

In [ ]:
batch_size = 8

dataloader_test = DataLoader(dataset_test, 
                              sampler=RandomSampler(dataset_test), 
                              batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
# Setting up optimizer
optimizer = AdamW(model.parameters(),
                  lr=5e-5, 
                  eps=1e-8)

In [ ]:
# Setting up scheduler
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_test)*epochs)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np

In [ ]:
def precision_score_func(labels, preds):
  labels_flat = labels.flatten()
  preds_flat = np.argmax(preds, axis=1).flatten()
  return precision_score(labels_flat, preds_flat, average='weighted')

def recall_score_func(labels, preds):
  labels_flat = labels.flatten()
  preds_flat = np.argmax(preds, axis=1).flatten()
  return recall_score(labels_flat, preds_flat, average='weighted')

def f1_score_func(labels, preds):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
def evaluate(dataloader_train):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_train:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_train_avg = loss_val_total/len(dataloader_train) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_train_avg, predictions, true_vals

In [ ]:
# Training loop
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
             
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    train_loss, predictions, true_vals = evaluate(dataloader_train)
    train_precision = precision_score_func(true_vals, predictions)
    train_recall = recall_score_func(true_vals, predictions)
    train_f1 = f1_score_func(true_vals, predictions)
    tqdm.write(f'Train loss: {train_loss}')
    tqdm.write(f'Precision Score (weighted): {train_precision}')
    tqdm.write(f'Recall Score (Weighted): {train_recall}')
    tqdm.write(f'F1 Score (Weighted): {train_f1}')

In [ ]:
model.load_state_dict(torch.load('model_name.model', map_location=torch.device('cuda')))
model = model.to(device)

In [ ]:
_, predictions, true_vals = evaluate(dataloader_train)

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)

In [ ]:
from sklearn import metrics

y_true = true_vals.flatten()
y_pred = np.argmax(predictions, axis=1).flatten()
print(metrics.confusion_matrix(y_true, y_pred))
print(metrics.classification_report(y_true, y_pred, digits=2))